# Grid2x2 using RLlib

In [1]:
import csv
import os
from typing import Optional

import ray
from ray.rllib.env.wrappers.multi_agent_env_compatibility import MultiAgentEnvCompatibility
from ray.tune.registry import register_env

from envs import MultiAgentSumoEnv
from observation import Cologne8ObservationFunction
from reward_functions import combined_reward

In [2]:
import random
import numpy as np
import torch

TEST_NUM = 1
SEED = 23423  # default SUMO seed no.
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

ENV_NAME = "cologne8"
OBS_CLASS = Cologne8ObservationFunction
assert not os.path.exists(os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}"))

In [3]:
def train_env_creator(args):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "begin_time": 25200,
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": OBS_CLASS,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(**env_params)
    return env

In [4]:
from ray.rllib.algorithms.ppo import PPOConfig

# From https://github.com/ray-project/ray/blob/master/rllib/tuned_examples/ppo/atari-ppo.yaml

train_env = MultiAgentEnvCompatibility(train_env_creator({}))

config: PPOConfig
config = (
    PPOConfig()
    .environment(env=ENV_NAME)
    .framework(framework="torch")
    .rollouts(
        rollout_fragment_length=100,
        num_rollout_workers=10,
    )
    .training(
        lambda_=0.95,
        kl_coeff=0.5,
        clip_param=0.1,
        vf_clip_param=10.0,
        entropy_coeff=0.01,
        train_batch_size=1000,
        sgd_minibatch_size=100,
        num_sgd_iter=10,
    )
    .evaluation(
        evaluation_duration=1,
        evaluation_num_workers=1,
        evaluation_sample_timeout_s=300,
    )
    .debugging(seed=SEED)
    .resources(num_gpus=int(os.environ.get("RLLIB_NUM_GPUS", "1")))
    .multi_agent(
        policies=set(train_env.env.ts_ids),
        policy_mapping_fn=(lambda agent_id, *args, **kwargs: agent_id),
    )
    .fault_tolerance(recreate_failed_workers=True)
)

delta_wait_time_reward 0.875


In [5]:
csv_dir = os.path.join("outputs",ENV_NAME,f"test_{TEST_NUM}")
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)

## Play Untrained Agent

In [6]:
def eval_env_creator(csv_path: Optional[str] = None, tb_log_dir: Optional[str] = None):
    env_params = {
        "net_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.net.xml"),
        "route_file": os.path.join("nets",ENV_NAME,f"{ENV_NAME}.rou.xml"),
        "begin_time": 25200,
        "num_seconds": 3600,
        "reward_fn": combined_reward,
        "sumo_seed": SEED,
        "observation_class": OBS_CLASS,
        "add_system_info": False,
    }
    congestion_reward = combined_reward.__defaults__[0].__name__
    alpha = combined_reward.__defaults__[1]  # congestion component coefficient
    print(congestion_reward, alpha)

    env = MultiAgentSumoEnv(eval=True, csv_path=csv_path, tb_log_dir=tb_log_dir, **env_params)
    return env

In [7]:
ray.init()

csv_path = os.path.join(csv_dir, "untrained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_untrained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure"])

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

2023-06-06 15:57:48,506	INFO worker.py:1625 -- Started a local Ray instance.


In [8]:
algo = config.build()

2023-06-06 15:57:52,855	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-06 15:57:52,889	INFO algorithm.py:527 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=21784) delta_wait_time_reward 0.875
(RolloutWorker pid=6072) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 16ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=24172) 2023-06-06 15:58:00,487	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-06 15:58:03,526	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=6444) delta_wait_time_reward 0.875 [repeated 10x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RolloutWorker pid=15576) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 20ms, vehicles TOT 0 ACT 0 BUF 0)                      [repeated 9x across cluster]


(RolloutWorker pid=6444) 2023-06-06 15:58:08,002	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-06 15:58:08,257	INFO trainable.py:172 -- Trainable.setup took 15.372 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [9]:
algo.evaluate()

(RolloutWorker pid=6444) 2023-06-06 16:01:23,564	WARNING env_runner_v2.py:154 -- More than 5760 observations in 720 env steps for episode 522901342373682868 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.


Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 320ms, vehicles TOT 2046 ACT 78 BUF 0)             s TOT 2 ACT 2 BUF 0)                  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 320ms, vehicles TOT 2046 ACT 78 BUF 0)             s TOT 2 ACT 2 BUF 0)                  


{'evaluation': {'episode_reward_max': -119341.47406456462,
  'episode_reward_min': -119341.47406456462,
  'episode_reward_mean': -119341.47406456462,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'247379907': -25146.52560635597,
   '252017285': -9602.611715454253,
   '256201389': -205.03901679367402,
   '26110729': -54602.845117381265,
   '280120513': -3965.094053775799,
   '32319828': -1839.3006509249046,
   '62426694': -3769.540023045584,
   'cluster_1098574052_1098574061_247379905': -20210.517880832806},
  'policy_reward_max': {'247379907': -25146.52560635597,
   '252017285': -9602.611715454253,
   '256201389': -205.03901679367402,
   '26110729': -54602.845117381265,
   '280120513': -3965.094053775799,
   '32319828': -1839.3006509249046,
   '62426694': -3769.540023045584,
   'cluster_1098574052_1098574061_247379905': -20210.517880832806},
  'policy_reward_mean': {'247379907': -25146.52560635597,
   '252017285': -9602.611715454

In [10]:
ray.shutdown()

## Train RL Agent

In [11]:
ray.init()

register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(train_env_creator(config)))

2023-06-06 16:01:31,129	INFO worker.py:1625 -- Started a local Ray instance.


In [12]:
algo = config.build()

2023-06-06 16:01:35,439	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=20048) delta_wait_time_reward 0.875
(RolloutWorker pid=20048) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 13ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=1732) 2023-06-06 16:01:43,234	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-06 16:01:43,748	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


(RolloutWorker pid=9260) delta_wait_time_reward 0.875 [repeated 10x across cluster]


(RolloutWorker pid=9260) 2023-06-06 16:01:48,259	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.


(RolloutWorker pid=9260) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 12ms, vehicles TOT 0 ACT 0 BUF 0)                      [repeated 10x across cluster]


2023-06-06 16:01:48,503	INFO trainable.py:172 -- Trainable.setup took 13.059 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [13]:
from datetime import datetime

TRAIN_EPS = 1400  # 720 * 1400 == 1_008_000 total timesteps
CHECKPOINT_FREQ = 100
assert TRAIN_EPS % CHECKPOINT_FREQ == 0

tic = datetime.now()

for i in range(TRAIN_EPS):
    results = algo.train()

    if (i+1) % CHECKPOINT_FREQ == 0:
        algo.save(os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}"))

toc = datetime.now()

2023-06-06 16:02:03,424	WARNING deprecation.py:50 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
(RolloutWorker pid=20048) Warning: Vehicle '137709_412_0' performs emergency braking on lane ':26110729_2_0' with decel=9.00, wished=4.50, severity=1.00, time=25932.00.


Step #25900.00 (0ms ?*RT. ?UPS, TraCI: 122ms, vehicles TOT 395 ACT 89 BUF 0)              es TOT 2 ACT 2 BUF 0)                  


(RolloutWorker pid=21792) Warning: Vehicle '188987_434_0' performs emergency braking on lane ':26110729_5_1' with decel=9.00, wished=4.50, severity=1.00, time=26154.00.


Step #26100.00 (1ms ~= 1000.00*RT, ~126000.00UPS, TraCI: 142ms, vehicles TOT 581 ACT 126 BCI: 49ms, vehicles TOT 2 ACT 2 BUF 0)  


(RolloutWorker pid=3060) Warning: Vehicle '153757_420_0' performs emergency braking on lane ':280120513_1_0' with decel=9.00, wished=4.50, severity=1.00, time=26282.00.


...

Step #26300.00 (1ms ~= 1000.00*RT, ~119000.00UPS, TraCI: 94ms, vehicles TOT 710 ACT 119 BUCI: 38ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 95ms, vehicles TOT 1953 ACT 67 BUF 93)             es TOT 2 ACT 2 BUF 0)                  
Step #28800.00 (1ms ~= 1000.00*RT, ~68000.00UPS, TraCI: 95ms, vehicles TOT 1958 ACT 68 BUFes TOT 2 ACT 2 BUF 0)                  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 91ms, vehicles TOT 1958 ACT 70 BUF 88)             I: 45ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 91ms, vehicles TOT 1954 ACT 72 BUF 92)             CI: 43ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~68000.00UPS, TraCI: 91ms, vehicles TOT 1958 ACT 68 BUFI: 42ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 91ms, vehicles TOT 1966 ACT 72 BUF 80)             CI: 40ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~73000.00UPS, TraCI: 94ms, vehicles TOT 1962 ACT 73 BUFI:

(RolloutWorker pid=3200) Warning: Vehicle '149857_417_0' performs emergency braking on lane ':26110729_7_0' with decel=9.00, wished=4.50, severity=1.00, time=27917.00.


Step #27900.00 (0ms ?*RT. ?UPS, TraCI: 85ms, vehicles TOT 1566 ACT 81 BUF 54)             s TOT 2 ACT 2 BUF 0)                  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 119ms, vehicles TOT 1958 ACT 66 BUF 88)            CI: 34ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~66000.00UPS, TraCI: 119ms, vehicles TOT 1953 ACT 66 BUCI: 37ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~67000.00UPS, TraCI: 120ms, vehicles TOT 1957 ACT 67 BUCI: 39ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~70000.00UPS, TraCI: 116ms, vehicles TOT 1955 ACT 70 BUI: 34ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (1ms ~= 1000.00*RT, ~72000.00UPS, TraCI: 104ms, vehicles TOT 1969 ACT 72 BUes TOT 2 ACT 2 BUF 0)                  
Step #28800.00 (1ms ~= 1000.00*RT, ~70000.00UPS, TraCI: 118ms, vehicles TOT 1965 ACT 70 BUI: 37ms, vehicles TOT 2 ACT 2 BUF 0)  
Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 113ms, vehicles TOT 1955 ACT 68 BUF 91)            I: 

(RolloutWorker pid=20048) Warning: Vehicle '248031_455_0' performs emergency braking on lane ':cluster_1098574052_1098574061_247379905_14_0' with decel=9.00, wished=4.50, severity=1.00, time=27637.00.


Step #27600.00 (1ms ~= 1000.00*RT, ~100000.00UPS, TraCI: 102ms, vehicles TOT 1428 ACT 100 CI: 31ms, vehicles TOT 2 ACT 2 BUF 0)  


In [14]:
str(toc - tic)

'7:27:00.987601'

In [15]:
ray.shutdown()

## Play Trained Agent

In [16]:
csv_path = os.path.join(csv_dir, "trained.csv")
tb_log_dir = os.path.join("logs", ENV_NAME, f"PPO_{TEST_NUM}", "eval_trained")

with open(csv_path, "a", newline="") as f:
    csv_writer = csv.writer(f)
    csv_writer.writerow(["sim_time", "arrived_num", "sys_tyre_pm", "sys_stopped",
                         "sys_total_wait", "sys_avg_wait", "sys_avg_speed",
                         "agents_tyre_pm", "agents_stopped", "agents_total_wait",
                         "agents_avg_speed", "agents_total_pressure"])

In [17]:
register_env(ENV_NAME, lambda config: MultiAgentEnvCompatibility(eval_env_creator(csv_path, tb_log_dir)))

In [18]:
ray.init()

2023-06-06 23:28:55,391	INFO worker.py:1625 -- Started a local Ray instance.


Python version:,3.8.16
Ray version:,2.4.0


In [19]:
from ray.rllib.algorithms.ppo import PPO

checkpoint_path = os.path.join("ray_checkpoints",ENV_NAME,f"test_{TEST_NUM}",f"checkpoint_{TRAIN_EPS:06}")
checkpoint_path = os.path.abspath(checkpoint_path)
print(checkpoint_path)

ppo_agent = PPO.from_checkpoint(checkpoint_path)

2023-06-06 23:28:58,928	WARNING checkpoints.py:109 -- No `rllib_checkpoint.json` file found in checkpoint directory c:\Users\admin\masters-proj\ray_checkpoints\cologne8\test_1\checkpoint_001400! Trying to extract checkpoint info from other files found in that dir.
2023-06-06 23:28:58,964	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.


c:\Users\admin\masters-proj\ray_checkpoints\cologne8\test_1\checkpoint_001400
(RolloutWorker pid=7800) delta_wait_time_reward 0.875
(RolloutWorker pid=7800) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 14ms, vehicles TOT 0 ACT 0 BUF 0)                     


(RolloutWorker pid=14640) 2023-06-06 23:29:06,689	WARNING env.py:285 -- Your MultiAgentEnv <MultiAgentEnvCompatibility instance> does not have some or all of the needed base-class attributes! Make sure you call `super().__init__()` from within your MutiAgentEnv's constructor. This will raise an error in the future.
2023-06-06 23:29:07,045	WARNING algorithm_config.py:784 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2023-06-06 23:29:11,173	INFO trainable.py:172 -- Trainable.setup took 12.205 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [20]:
ppo_agent.evaluate()

(RolloutWorker pid=2156) Warning: Vehicle '188987_434_0' performs emergency braking on lane '186623965#9_1' with decel=9.00, wished=4.50, severity=1.00, time=26154.00.
(RolloutWorker pid=2156) Warning: Vehicle '188987_434_0' performs emergency braking on lane '186623965#9_1' with decel=9.00, wished=4.50, severity=1.00, time=26154.00.


Step #26100.00 (1ms ~= 1000.00*RT, ~92000.00UPS, TraCI: 67ms, vehicles TOT 553 ACT 92 BUF I: 24ms, vehicles TOT 2 ACT 2 BUF 0)  
(RolloutWorker pid=2156) delta_wait_time_reward 0.875 [repeated 10x across cluster]
(RolloutWorker pid=2156) Step #0.00 (0ms ?*RT. ?UPS, TraCI: 10ms, vehicles TOT 0 ACT 0 BUF 0)                      [repeated 10x across cluster]


(RolloutWorker pid=2156) 2023-06-06 23:31:32,631	WARNING env_runner_v2.py:154 -- More than 5760 observations in 720 env steps for episode 522901342373682868 are buffered in the sampler. If this is more than you expected, check that that you set a horizon on your environment correctly and that it terminates at some point. Note: In multi-agent environments, `rollout_fragment_length` sets the batch size based on (across-agents) environment steps, not the steps of individual agents, which can result in unexpectedly large batches.Also, you may be waiting for your Env to terminate (batch_mode=`complete_episodes`). Make sure it does at some point.


Step #28800.00 (0ms ?*RT. ?UPS, TraCI: 277ms, vehicles TOT 1963 ACT 71 BUF 83)            s TOT 625 ACT 105 BUF 36)             


{'evaluation': {'episode_reward_max': -81932.39211473051,
  'episode_reward_min': -81932.39211473051,
  'episode_reward_mean': -81932.39211473051,
  'episode_len_mean': 720.0,
  'episode_media': {},
  'episodes_this_iter': 1,
  'policy_reward_min': {'247379907': -17761.701438447875,
   '252017285': -4692.316757143753,
   '256201389': -120.30639819466441,
   '26110729': -42791.40501786032,
   '280120513': -1826.5475681450803,
   '32319828': -1486.911637067086,
   '62426694': -1796.6097063459954,
   'cluster_1098574052_1098574061_247379905': -11456.593591525916},
  'policy_reward_max': {'247379907': -17761.701438447875,
   '252017285': -4692.316757143753,
   '256201389': -120.30639819466441,
   '26110729': -42791.40501786032,
   '280120513': -1826.5475681450803,
   '32319828': -1486.911637067086,
   '62426694': -1796.6097063459954,
   'cluster_1098574052_1098574061_247379905': -11456.593591525916},
  'policy_reward_mean': {'247379907': -17761.701438447875,
   '252017285': -4692.316757143

In [21]:
ray.shutdown()